# Semantic functions

### Initialize the kernel

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"

#!import Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Settings.InitializeKernel();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha

### ✍️ Create a simple plain semantic function

In [4]:
var promptTemplate = "When is the next total solar eclipse?";

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);
var response = await kernel.InvokeAsync(semanticFunction);

display(response.GetValue<string>())

As of my last update in early 2023, the next total solar eclipse is expected to occur on April 8, 2024. The path of totality for this eclipse will cross through Mexico, the United States, and Canada. It is noteworthy for being the next total solar eclipse to follow the August 21, 2017, event, which was dubbed the "Great American Eclipse."

During a total solar eclipse, the Moon passes directly between the Earth and the Sun, completely obscuring the Sun's disk and momentarily turning day into night for observers located within the narrow path of totality. The duration of totality—when the Sun's disk is completely covered—varies depending on the location within the path; for the April 8, 2024 eclipse, it can last up to approximately 4 minutes and 28 seconds at the point of greatest duration.

Viewers outside the path of totality may still see a partial solar eclipse, where the Sun is only partially covered by the Moon.

When planning to observe a solar eclipse, it's crucial to protect yo

### 📋 Now let's create a more complete semantic function (with input variable and execution settings)

In [6]:
var promptTemplate = "When is the next total {{$input}} eclipse? Respond only with JSON.";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 500,
    Temperature = 0.1,
};

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate, executionSettings);
var response = await kernel.InvokeAsync(semanticFunction, new() { ["input"] = "lunar" });

display(response.GetValue<string>())

```json
{
  "next_total_lunar_eclipse": "November 8, 2022"
}
```

### 📋 But what about if we have more input variables?

In [7]:
var promptTemplate = "When is the next total {{$input}} eclipse? Today is {{$today}}. Respond only with JSON.";

var executionSettings = new OpenAIPromptExecutionSettings 
{
     MaxTokens = 500,
     Temperature = 0.1 
};
var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "solar",
    ["today"] = "22st of April 2024"
};

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);
var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())

```json
{
  "next_total_solar_eclipse": {
    "date": "2024-04-08"
  }
}
```